#Import and initialization:

In [ ]:
from google.colab import drive
import sys
import os
import pandas as pd
import numpy as np
drive.mount('/content/drive')
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
import pickle
path = '/content/drive/MyDrive/Semantics In Intelligent Information Access/'
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [2]:
#embedding_type = 'exnovo'
embedding_type = 'stored'
dataset = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/dataset_covered_recipes.csv",sep=';')
dataset_sustainble =  dataset.loc[np.where(dataset['SUSTAINABILITY'].eq('HIGH'))]
dataset_sustainble.reset_index(inplace = True,drop = True)
dataset_mid_sustainble =  dataset.loc[np.where(dataset['SUSTAINABILITY'].eq('MEDIUM'))]
dataset_mid_sustainble.reset_index(inplace = True,drop = True)

if(embedding_type == 'exnovo'):  
  embedding_sus_recipes = model.encode(dataset_sustainble['ingredients'])
  embedding_mid_sus_recipes = model.encode(dataset_mid_sustainble['ingredients'])
  with open(path+'embedding_sus_recipes.pkl', "wb") as fOut:
    pickle.dump({'sentences': dataset_sustainble['ingredients'], 'embeddings': embedding_sus_recipes}, fOut, protocol=pickle.HIGHEST_PROTOCOL)
  with open(path+'embedding_mid_sus_recipes.pkl', "wb") as fOut:
    pickle.dump({'sentences': dataset_mid_sustainble['ingredients'], 'embeddings': embedding_mid_sus_recipes}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

if(embedding_type == 'stored'):
  with open(path+'embedding_sus_recipes.pkl', "rb") as fIn:
    stored_data_sus_recipes = pickle.load(fIn)
    embedding_sus_recipes = stored_data_sus_recipes['embeddings']
  with open(path+'embedding_mid_sus_recipes.pkl', "rb") as fIn:
    stored_data_mid_sus_recipes = pickle.load(fIn)
    embedding_mid_sus_recipes = stored_data_mid_sus_recipes['embeddings']

#Test

In [3]:
my_ingredients = 'tomato,zucchini,cucumber,meat'
embedding_ing = model.encode(my_ingredients)
sus_scores = []  
sus_scores = util.pytorch_cos_sim(embedding_ing, embedding_sus_recipes).flatten()
print(sus_scores)
sus_sorted = sus_scores.argsort()[-3:]
print(sus_scores[sus_sorted])
sus_results = dataset_sustainble.loc[sus_sorted]
sus_results

tensor([0.1262, 0.2792, 0.2792,  ..., 0.4645, 0.4537, 0.4869])
tensor([0.7346, 0.7523, 0.7733])


,recipe_id,title,url,ingredients,SUSTAINABILITY
5121,23084,Pineapple-Jalapeno Salsa,http://www.food.com/recipe/pineapple-jalapeno-...,"tomato,cucumber,pineapple juice,jalapeno peppe...",HIGH
64,15894,Assorted Relishes Recipe,http://cookeatshare.com/recipes/assorted-relis...,"tomato,radish,cucumber",HIGH
2057,34238,Broccoli Salad,http://www.food.com/recipe/broccoli-salad-174615,"tomato,olives,broccoli,cucumber,onion,carrot,s...",HIGH


In [4]:
mid_sus_scores = []  
mid_sus_scores = util.pytorch_cos_sim(embedding_ing, embedding_mid_sus_recipes).flatten()
print(mid_sus_scores)
mid_sus_sorted = mid_sus_scores.argsort()[-3:]
print(mid_sus_scores[mid_sus_sorted])
mid_sus_results = dataset_mid_sustainble.loc[mid_sus_sorted]
mid_sus_results

tensor([0.4190, 0.4209, 0.4424,  ..., 0.4928, 0.4818, 0.6035])
tensor([0.8194, 0.8210, 0.8218])


,recipe_id,title,url,ingredients,SUSTAINABILITY
14236,2764,20-Minute Nachos,http://www.kraftrecipes.com/recipes/20-minute-...,"tomato,ceddar,pepper,zucchini",MEDIUM
3372,2614,Harvested Chicken Stew,http://allrecipes.com/recipe/harvested-chicken...,"tomato,onion,corn canned,pea,celery,chicken bo...",MEDIUM
3148,27254,Stir Fry Turkey with Veggies,http://www.food.com/recipe/stir-fry-turkey-wit...,"tomato,olive oil,turkey meat with bone,green b...",MEDIUM


In [5]:
sus_max_score = sus_scores[sus_sorted][-1]
sus_max_index = sus_sorted[-1]
mid_sus_max_score = mid_sus_scores[mid_sus_sorted][-1]
mid_sus_max_index = mid_sus_sorted[-1]
if(sus_max_score > 0.50):
  sus_recommendation = 'If you want to eat a relly sustainable dish using your ingredients I suggest you to eat the '+dataset_sustainble.loc[[sus_max_index]]['title'].values[0]+' recipe.'
  sus_url_recommendation = 'You can read more information on the recipe at '+dataset_sustainble.loc[[sus_max_index]]['url'].values[0]
if(mid_sus_max_score > 0.50):
  mid_sus_recommendation = 'If you want to eat a medium sustainable dish using your ingredients I suggest you to eat the '+dataset_mid_sustainble.loc[[mid_sus_max_index]]['title'].values[0]+' recipe.'
  mid_sus_url_recommendation = 'You can read more information on the recipe at '+dataset_mid_sustainble.loc[[mid_sus_max_index]]['url'].values[0]  

print(mid_sus_recommendation)
print(mid_sus_url_recommendation)
print(' ')
print(sus_recommendation)
print(sus_url_recommendation)


If you want to eat a medium sustainable dish using your ingredients I suggest you to eat the Stir Fry Turkey with Veggies recipe.
You can read more information on the recipe at http://www.food.com/recipe/stir-fry-turkey-with-veggies-37813
 
If you want to eat a relly sustainable dish using your ingredients I suggest you to eat the Broccoli Salad recipe.
You can read more information on the recipe at http://www.food.com/recipe/broccoli-salad-174615
